In [12]:
import pandas as pd
import chardet

# check the encoding
with open("HW1data.csv", "rb") as police_data:
    result = chardet.detect(police_data.read(100000))
    enc = result["encoding"]
    # print(enc)

df = pd.read_csv(r"HW1data.csv", encoding=enc)
print(df.shape)
# get rid of all rows with the following region being Nan
# df = df.dropna(axis=0, how="any", subset=["PatientId", "ScheduledDay", "AppointmentDay", "SMS_received", "No-show"])
# parsing the datetime


# The process of collecting msg of patients
tmp = df.set_index("PatientId")
tmp = tmp.groupby(level = 0).bfill().reset_index().drop_duplicates(subset=["PatientId"])
print(tmp)

# The process of updating data back to the dataframe
tmp = tmp.set_index("PatientId")
df = df.set_index("PatientId")
df.update(tmp, overwrite=True)
df = df.reset_index()
print(len(df))
print(df.isnull().sum())
# print(df)


# get rid of the following rows that are still containing nan
df = df.dropna(axis=0, how="any", subset=["PatientId",
                                          "Gender", "Age", "Neighbourhood", "ScheduledDay", "AppointmentDay",
                                          "SMS_received", "No-show"])

df["ScheduledDay"] = pd.to_datetime(df["ScheduledDay"], format="%Y-%m-%dT%H:%M:%SZ")
df["AppointmentDay"] = pd.to_datetime(df["AppointmentDay"], format="%Y-%m-%dT%H:%M:%SZ")
# delete col PatientId AppointmentID
print(len(df))
df = df.drop(["PatientId", "AppointmentID"], axis=1)

# fill na with 0 for Scholarship, Hipertension ,Diabetes ,Alcoholism, Handcap
df["Scholarship"] = df["Scholarship"].fillna(0)
df["Hipertension"] = df["Hipertension"].fillna(0)
df["Alcoholism"] = df["Alcoholism"].fillna(0)
df["Handcap"] = df["Handcap"].fillna(0)
df["Diabetes"] = df["Diabetes"].fillna(0)

# until now ,there is no nan in the dataframe
# print(pd.DataFrame.isnull(df).sum())

# delete rows with age problem (<0)
bad_rows = df.loc[df["Age"] < 0]
df = df.drop(bad_rows.index)
# print(df)


# computing delta day
delta_date = df["AppointmentDay"] - df["ScheduledDay"]
df["Delta_Day"] = delta_date.dt.days

# deleting rows that have Delta_Day<0
df = df.drop(df.loc[df["Delta_Day"] < 0].index)
df = df.reset_index(drop=True)

# judging the week days
wd = ["Monday", "Tuesday", "Wednesday", "Thusday", "Friday", "Saturday", "Sunday"]
week_of_day_ScheduledDay = df["ScheduledDay"].dt.weekday
week_of_day_AppointmentDay = df["AppointmentDay"].dt.weekday
week_of_day_ScheduledDay = [wd[x] for x in week_of_day_ScheduledDay]
week_of_day_AppointmentDay = [wd[x] for x in week_of_day_AppointmentDay]
# print(week_of_day_AppointmentDay)
df["SDay_DOW"] = week_of_day_ScheduledDay
df["ADay_DOW"] = week_of_day_AppointmentDay
# print(df)

# delete col ScheduledDay, AppointmentDay
df = df.drop(["ScheduledDay", "AppointmentDay"], axis=1)

# reset index and turning dtype of Age to int
df = df.reset_index(drop=True)
df["Age"] = [int(x) for x in df["Age"]]


# df is the final result of the homework
print(df)

# write to result.csv
# df.to_csv("result.csv", index=None)

(667536, 14)
           PatientId  AppointmentID Gender          ScheduledDay  \
0       4.838649e+14      5751504.0      F  2016-05-31T10:14:47Z   
1       4.858271e+13      5736107.0      F  2016-05-25T07:55:55Z   
2       7.156662e+12      5708125.0      F  2016-05-17T10:39:38Z   
3       8.816117e+14      6313735.0      F  2017-03-30T05:52:45Z   
4       1.158164e+14      6261424.0      F  2016-12-16T23:34:44Z   
...              ...            ...    ...                   ...   
559586  1.739789e+12      5689034.0      F  2016-05-12T07:57:22Z   
565308  5.385329e+13      5719452.0      F  2016-05-19T11:19:28Z   
577913  5.756261e+12      5453021.0      F  2016-03-10T10:33:16Z   
579200  1.818381e+13      6243116.0      F  2017-01-11T09:48:38Z   
615587  9.499292e+12      5569965.0      F  2016-04-12T07:18:54Z   

              AppointmentDay   Age        Neighbourhood  Scholarship  \
0       2016-06-08T00:00:00Z  21.0  ILHA DE SANTA MARIA          0.0   
1       2016-05-25T00:00:0

In [7]:
import numpy as np
a = pd.to_datetime(np.NaN, format="%Y-%m-%dT%H:%M:%SZ")
print(a)
print(a == np.NaN)

NaT
False
